# Strategy backtesting example

This is a notebook that runs through an example of a trading strategy based on moving averages.

When the short term moving average crosses above the longer term moving average we buy
When the short term moving average crosses below the longer term moving average we sell

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import os
import requests
import sys
import boto3
import yfinance as yf
import pandas as pd

In [8]:
def get_data(ticker="AAPL"):
    data = yf.download(ticker, timeout=120)
    df = pd.DataFrame(data['Adj Close'][ticker])
    df.rename(columns={ticker: "Close"}, inplace=True)
    # The strategy only uses Adjusted Close, but the library requires Open, High, and Low
    df['Open'] = df['High'] = df['Low'] = df['Close']
    df = df.reindex(['Open', 'High', 'Low', 'Close'], axis=1)
    return df

data = get_data("AAPL")
data['Close']

[*********************100%***********************]  1 of 1 completed


Date
1980-12-12      0.098834
1980-12-15      0.093678
1980-12-16      0.086802
1980-12-17      0.088951
1980-12-18      0.091530
                 ...    
2024-12-03    242.649994
2024-12-04    243.009995
2024-12-05    243.039993
2024-12-06    242.839996
2024-12-09    246.750000
Name: Close, Length: 11090, dtype: float64

In [6]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

ticker="MSFT"
data = get_data(ticker)
bt = Backtest(data, SmaCross,
            cash=10000, commission=.002,
            exclusive_orders=True)

output = bt.run()
output
   

[*********************100%***********************]  1 of 1 completed


Start                     1986-03-13 00:00:00
End                       2024-12-09 00:00:00
Duration                  14151 days 00:00:00
Exposure Time [%]                   99.784924
Equity Final [$]                  22646.78974
Equity Peak [$]                  79582.437014
Return [%]                         126.467897
Buy & Hold Return [%]           745419.470862
Return (Ann.) [%]                    2.132133
Volatility (Ann.) [%]               33.701129
Sharpe Ratio                         0.063266
Sortino Ratio                        0.095227
Calmar Ratio                         0.027907
Max. Drawdown [%]                  -76.401079
Avg. Drawdown [%]                  -10.884643
Max. Drawdown Duration     6096 days 00:00:00
Avg. Drawdown Duration      227 days 00:00:00
# Trades                                  499
Win Rate [%]                         40.08016
Best Trade [%]                     140.879262
Worst Trade [%]                    -23.461847
Avg. Trade [%]                    

In [7]:
bt.plot()

Row(id='1392', ...)